# ITEM OUTLET SALES PREDICTION

In [ ]:
#IMPORTING LIBRARIES:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats

In [ ]:
#Importing Dataset:
data_shopping= pd.read_csv("../input/big-mart-sales-prediction/Train.csv")

In [ ]:
data_shopping.head()

In [ ]:
#Making a Copy of Original Data
data =data_shopping.copy()

In [ ]:
#Lowering the down column names
data.columns = data.columns.str.lower()

In [ ]:
#Calculating Missing Values
(data.isnull().sum()/len(data))*100

## Imputation Of Missing Values in the Columns:

In [ ]:
group_mean_weight = data.pivot_table(index = ["item_type"], values = "item_weight", aggfunc = [np.mean])

In [ ]:
group_mean_weight

In [ ]:
mean_weight = group_mean_weight.iloc[:,[0][0]]

In [ ]:
mean_weight

In [ ]:
# Function to impute Missing Value in item_weight column:

def missing_value(cols):
    item_type = cols[0]
    item_weight =cols[1]
    if pd.isnull(item_weight):
        if item_type == "Baking Goods":
            return 12.277
        elif item_type == "Breads":
            return 11.347
        elif item_type == "Breakfast":
            return 12.768
        elif item_type == "Canned":
            return 12.30
        elif item_type == "Dairy":
            return 13.42
        elif item_type == "Frozen Foods":
            return  12.867061
        elif item_type == "Fruits and Vegetables":
            return 13.224769
        elif item_type == "Hard Drinks":
            return 11.400328
        elif item_type == "Health and Hygiene":
            return 13.142314
        elif item_type == "Household":
            return 13.384736
        elif item_type == "Meat":
            return 12.817344
        elif item_type == "Others":
            return 13.853285
        elif item_type == "Seafood":
            return 12.552843
        elif item_type == "Snack Foods":
            return 12.987880
        elif item_type == "Soft Drinks":
            return 11.847460
        elif item_type == "Starchy Foods":
            return 13.690731
    return item_weight   
        
       
            
            
            
            

In [ ]:
#Imputing the missing value by using defined function
data["item_weight"] = data[["item_type","item_weight"]].apply(missing_value, axis = 1)

In [ ]:
data.head()

## DATA UNDERSTANDING THROUGH VISUALIZATION

In [ ]:
#FINDING FREQUENCY COUNT OF OUTLET TYPE
sns.countplot(data = data, x = "outlet_type",hue = "outlet_size")
plt.xticks(rotation =90)

In [ ]:
# Function for Imputing Missing value in Outlet_Size column:

def impute_size(cols):
    size = cols[0]
    ot_type = cols[1]
    if pd.isnull(size):
        if ot_type == "Supermarket Type1":
            return "Small"
        elif ot_type == "Supermarket Type2":
            return "Medium"
        elif ot_type == "Grocery Store":
            return "Small"
        elif ot_type == "Supermarket Type3":
            return "Medium"
    return size    

In [ ]:
#USING ABOVE DEFINED FUNCTION IMPUTE MISSING VALUES IN OUTLET SIZE COLUMNS
data["outlet_size"] = data[["outlet_size","outlet_type"]].apply(impute_size, axis = 1)

**While exploring the item_fat_content, we founded "4" different unique values.**
Following ARE:
low fat - represent LOW FAT
reg - represent regular

In [ ]:
data["item_fat_content"].unique()

In [ ]:
data["item_fat_content"] = data["item_fat_content"].str.replace("LF", "low fat").str.replace("reg", "regular").str.lower()

In [ ]:
data["item_fat_content"].unique()

In [ ]:
data.head()

---
While exploring the **"item_visibility"** column, We Found many items visibility as **0.0** which is impossible. Hence, imputing these **0.0** values with **MEAN** of respective item of visibility.

In [ ]:
mean_visibility = data.pivot_table(index = "item_identifier",  values = "item_visibility")

In [ ]:
data.loc[(data["item_visibility"] == 0.0), "item_visibility"] = data.loc[(data["item_visibility"] == 0.0), "item_identifier"].apply(lambda x : mean_visibility.at[x, "item_visibility"])
                                                                                        

In [ ]:
#understanding outlet_identifier depending on item_outlet_sales
import matplotlib.pyplot as plt
plt.figure(figsize = (10,5))
sns.barplot(data = data, x = "outlet_identifier", y= "item_outlet_sales")

In [ ]:
#Understanding to ultet_establishment per year with respective to mean of each respective year item outlet sales
data.groupby("outlet_establishment_year")["item_outlet_sales"].mean().plot.bar()
plt.ylabel("Mean of Item outlet sales")

In [ ]:
#Understanding to item_type per year with respective to mean of each respective year item outlet sales
data.groupby("item_type")["item_outlet_sales"].mean().plot.bar()
plt.ylabel("Mean of Item outlet sales")

In [ ]:
#Understanding to outlet_type per year with respective to mean of each respective year item outlet sales
data.groupby("outlet_type")["item_outlet_sales"].mean().plot.bar()
plt.ylabel("Mean of Item outlet sales")

In [ ]:
data.head()

# Replacing The Categorical  Values Containing Columns With Their Respective Frequency or Count

In [ ]:
cols = ['item_identifier', 'item_fat_content',
       'item_type', 'outlet_identifier',
       'outlet_establishment_year', 'outlet_size', 'outlet_location_type',
       'outlet_type']

In [ ]:
#MAPPING EACH CATEGORICAL COLUMN WITH RESPECTIVE FREQUENCY OF THE VALUES IN THE COLUMNS
for i in cols:
    x  = data[i].value_counts().to_dict()
    data[i] = data[i].map(x)

In [ ]:
#RESULTING DATASET AFTER CATEGORICAL VALUES CONVERTED TO NUMERICAL COLUMN
data.head()

In [ ]:
#COPYING DATA 
new_data= data.copy()

In [ ]:
#FINDING CORRELATION BETWEEN EACH COLUMNS BY USING HEATMAP
plt.figure(figsize = (10,5))
sns.heatmap(new_data.corr(), cmap = "RdYlGn", annot = True)

In [ ]:
#REMOVING LESS CORRELATED COLUMNS 
new_data =new_data.drop(["item_weight","item_identifier", "item_type", "item_fat_content","outlet_location_type"], axis = 1)

In [ ]:
new_data.head()

In [ ]:
#CALCULATING THE SKEWNESS OF THE DATA
new_data.skew()

### VISUALIZING THE SKEWNESSS OF THE DATASET

In [ ]:
# Before Transformation
fig, ax = plt.subplots(4,2,figsize = (15,15))
sns.distplot(new_data["item_visibility"], kde =True, ax=ax[0,0], color = "red")
sns.distplot(new_data["item_mrp"], kde =True, ax=ax[0,1], color = "blue")
sns.distplot(new_data["outlet_identifier"], kde =True, ax=ax[1,0], color = "orange")
sns.distplot(new_data["outlet_type"], kde =True, ax=ax[1,1], color = "black")
sns.distplot(new_data["outlet_size"], kde =True, ax=ax[2,0], color = "magenta")
sns.distplot(new_data["outlet_establishment_year"], kde =True, ax=ax[3,0])
sns.distplot(new_data["item_outlet_sales"], kde =True, ax=ax[3,1])



In [ ]:
new_data.columns

### From the above plot it is clear that many columns are not following the normal distribution and not properly scaled. Therefore, we will apply "Logarithmic" Transformation on above dataset to make it scale and reduce the skewness.

In [ ]:
for i in new_data.columns:
    new_data[i] =np.log(new_data[i])

In [ ]:
new_data.head()

In [ ]:
# After Transformation
fig, ax = plt.subplots(4,2,figsize = (15,15))
sns.distplot(new_data["item_visibility"], kde =True, ax=ax[0,0], color = "red")
sns.distplot(new_data["item_mrp"], kde =True, ax=ax[0,1], color = "blue")
sns.distplot(new_data["outlet_identifier"], kde =True, ax=ax[1,0], color = "orange")
sns.distplot(new_data["outlet_type"], kde =True, ax=ax[1,1], color = "black")
sns.distplot(new_data["outlet_size"], kde =True, ax=ax[2,0], color = "magenta")
sns.distplot(new_data["outlet_establishment_year"], kde =True, ax=ax[3,0])
sns.distplot(new_data["item_outlet_sales"], kde =True, ax=ax[3,1])

In [ ]:
new_data.skew()

# Fitting Linear Regression Model:

In [ ]:
#Independent Variables:
x = new_data.drop("item_outlet_sales", axis = 1) 

#Depenedent Variables 
y = new_data["item_outlet_sales"].values.reshape(-1,1)


In [ ]:
#Splitting The data  into Train and Test Dataset:
from sklearn.model_selection import train_test_split
x_train,x_test, y_train, y_test = train_test_split(x,y, test_size =0.20, random_state = 3)

In [ ]:
#Applying Linear Regression Model
from sklearn.linear_model import LinearRegression
regressor =LinearRegression()
regressor.fit(x_train, y_train)

In [ ]:
#Prediction
y_pred = regressor.predict(x_test)

In [ ]:
#Accuracy of Model (Apply R2_score)
from sklearn.metrics import r2_score, mean_squared_error
r2_score(y_test, y_pred)

In [ ]:
#Checking Root Mean Square error
from math import sqrt
rmse = sqrt(mean_squared_error(y_test,  y_pred))
rmse

# 1. R2_Score = 75.70%
# 2. RMSE  = 0.519

In [ ]:
#Residue of the Linear Regression Model 
residue_lr = y_test -y_pred

In [ ]:
#Plotting Residual Plot
plt.scatter(y_test,residue_lr, c = "red")
plt.xlabel("residual")
plt.ylabel("y_test")
plt.axhline(y = 0)

# Regularization:

### 1. **LASSO Regression**:

In [ ]:
#Importing LASSO AND RIDGE from sklearn library:
#Apply Lasso Regularization Technique
from sklearn.linear_model import Lasso, Ridge
ls = Lasso(alpha = 0.009)
ls.fit(x_train, y_train)


In [ ]:
#prediction by LASSO model
ls_pred = ls.predict(x_test)

In [ ]:
#Accuracy After Lasso(by R2_score)
r2_score(y_test,ls_pred)

In [ ]:
#Root Mean Square Error
rmse_lasso = sqrt(mean_squared_error(y_test, ls_pred))
rmse_lasso

# AFTER LASSO REGRESSION
# 1.R2_SCORE= 74.68%
# 2.RMSE = 0.53

In [ ]:
#Getting Lasso Coefficent
lasso_coeff = pd.Series(ls.coef_, index =x.columns) 

In [ ]:
#Visualization of Coefficent after LASSO 
lasso_coeff.plot(kind = "bar")

In [ ]:
#RESHAPING THE PREDICTED VALUES
ls_pred= ls_pred.reshape(-1,1)

ls_pred

In [ ]:
#RESIDUE VALUE AFTER LASSO REGRESSION
residue = y_test - ls_pred

In [ ]:
plt.scatter(y_test.reshape(-1,1),residue, c = "red")
plt.xlabel("residual")
plt.ylabel("y_test")
plt.axhline(y = 0)

## 2. **RIDGE Regression**:


In [ ]:
#Ridge Regression
rr = Ridge(alpha = 0.009)
rr.fit(x_train, y_train)

In [ ]:
#Prediction AFTER Ridge regression
rr_pred = rr.predict(x_test)

In [ ]:
#Accuracy score check
r2_score(y_test, y_pred)

In [ ]:
#RMSE
rmse_ridge = sqrt(mean_squared_error(y_test, rr_pred))
rmse_ridge

# AFTER RIDGE REGRESSION
# 1. R2_SCORE = 75.57%
# 2. RMSE = 0.519

In [ ]:
#residue after ridge
residue_rr = y_test-rr_pred

In [ ]:
#plotting of residual graph after RIDGE REGRESSION
plt.scatter(y_test,residue_rr, c = "red")
plt.xlabel("residual")
plt.ylabel("y_test")
plt.axhline(y = 0)

# Selcting The Better Model

## 1.From above three different model after applying LASSO AND RIDGE Regression,  we can accept the model which is giving the better  "**RMSE SCORE**" and higher accuracy (R2_SCORE). 

## 2. Lasso Regression is showing the low accuracy in terms of Linear regression.
## 3. Ridge Regression is showing the similar accuracy in terms of Linear Regression.

### HENCE, ACCEPTING THE LINEAR REGRESSION MODEL TO PREDICT THE TEST DATA ITEM OUTLET SALE.


# LOADING TEST DATA

In [ ]:
test_data = pd.read_csv("../input/big-mart-sales-prediction/Test.csv")

In [ ]:
test_data.head()

In [ ]:
test = test_data.copy()

In [ ]:
#LOWERING THE COLUMNS NAMES 
test.columns = test.columns.str.lower()

In [ ]:
#Calculating Missing Values
(test.isnull().sum()/len(test))*100

### IMPUTATION OF MISSING VALUES

In [ ]:
#calculating the mean of item_weight with respective to item_type
group_mean = test.pivot_table(index = ["item_type"], values = "item_weight", aggfunc = [np.mean])

In [ ]:
group_mean

In [ ]:
mean_weigh = group_mean.iloc[:,[0][0]]

In [ ]:
mean_weigh

In [ ]:
# Function to impute Missing Value in item_weight column:

def missing_value1(cols):
    item_type = cols[0]
    item_weight =cols[1]
    if pd.isnull(item_weight):
        if item_type == "Baking Goods":
            return 12.277
        elif item_type == "Breads":
            return 10.86
        elif item_type == "Breakfast":
            return  13.759603
        elif item_type == "Canned":
            return 12.393565
        elif item_type == "Dairy":
            return 12.955040
        elif item_type == "Frozen Foods":
            return  12.101543
        elif item_type == "Fruits and Vegetables":
            return 13.146659
        elif item_type == "Hard Drinks":
            return 11.844417
        elif item_type == "Health and Hygiene":
            return 13.216929
        elif item_type == "Household":
            return 13.270504
        elif item_type == "Meat":
            return 12.702148
        elif item_type == "Others":
            return 14.009725
        elif item_type == "Seafood":
            return 13.241136
        elif item_type == "Snack Foods":
            return 12.684256
        elif item_type == "Soft Drinks":
            return 11.691965
        elif item_type == "Starchy Foods":
            return 13.618247
    return item_weight  

In [ ]:
#applying the above function to the dataset
test["item_weight"] = test[["item_type","item_weight"]].apply(missing_value1, axis = 1)

In [ ]:
#Frequency count of the outlet_type 
sns.countplot(data = test, x = "outlet_type",hue = "outlet_size")
plt.xticks(rotation =90)

In [ ]:
# Function for Imputing Missing value in Outlet_Size column:

def impute_size1(cols):
    size = cols[0]
    ot_type = cols[1]
    if pd.isnull(size):
        if ot_type == "Supermarket Type1":
            return "Small"
        elif ot_type == "Supermarket Type2":
            return "Medium"
        elif ot_type == "Grocery Store":
            return "Small"
        elif ot_type == "Supermarket Type3":
            return "Medium"
    return size    

In [ ]:
test["outlet_size"] = test[["outlet_size","outlet_type"]].apply(impute_size1, axis = 1)

In [ ]:
test["item_fat_content"].unique()

In [ ]:
test["item_fat_content"] = test["item_fat_content"].str.replace("LF", "low fat").str.replace("reg", "regular").str.lower()

In [ ]:
test["item_fat_content"].unique()

In [ ]:
mean_item_visibility = test.pivot_table(index = "item_identifier",  values = "item_visibility")

In [ ]:
mean_item_visibility.head()

In [ ]:
test_d = test.copy()

### IMPUTING CATEGORICAL VALUES WITH FREQUENCY OF RESPECTIVE COLUMNS IN THE TEST DATA

In [ ]:
columns = ['item_identifier', 'item_fat_content',
       'item_type', 'outlet_identifier',
       'outlet_establishment_year', 'outlet_size', 'outlet_location_type',
       'outlet_type']
for i in columns:
    x  = test_d[i].value_counts().to_dict()
    test_d[i] = test_d[i].map(x)

In [ ]:
test_d.head()

In [ ]:
new_test_data = test_d.copy()

In [ ]:
new_test_data =new_test_data.drop(["item_weight","item_identifier", "item_type", "item_fat_content","outlet_location_type"], axis = 1)

In [ ]:
new_test_data.head()

In [ ]:
#CHECKING THE SKEWNESS OF THE TEST DATA
new_test_data.skew()

In [ ]:
#VISUALIZING THE SKEWNESS OF THE DATASET
# Before Transformation
fig, ax = plt.subplots(3,2,figsize = (15,15))
sns.distplot(new_test_data["item_visibility"], kde =True, ax=ax[0,0], color = "red")
sns.distplot(new_test_data["item_mrp"], kde =True, ax=ax[0,1], color = "blue")
sns.distplot(new_test_data["outlet_identifier"], kde =True, ax=ax[1,0], color = "orange")
sns.distplot(new_test_data["outlet_type"], kde =True, ax=ax[1,1], color = "black")
sns.distplot(new_test_data["outlet_size"], kde =True, ax=ax[2,0], color = "magenta")
sns.distplot(new_test_data["outlet_establishment_year"], kde =True, ax=ax[2,1])



In [ ]:
new_test_data.columns

In [ ]:
#Applying the log transformation
for i in new_test_data.columns:
    new_test_data[i] =np.log(new_test_data[i]+1)

In [ ]:
#After Transformation 
fig, ax = plt.subplots(3,2,figsize = (15,15))
sns.distplot(new_test_data["item_visibility"], kde =True, ax=ax[0,0], color = "red")
sns.distplot(new_test_data["item_mrp"], kde =True, ax=ax[0,1], color = "blue")
sns.distplot(new_test_data["outlet_identifier"], kde =True, ax=ax[1,0], color = "orange")
sns.distplot(new_test_data["outlet_type"], kde =True, ax=ax[1,1], color = "black")
sns.distplot(new_test_data["outlet_size"], kde =True, ax=ax[2,0], color = "magenta")
sns.distplot(new_test_data["outlet_establishment_year"], kde =True, ax=ax[2,1])


In [ ]:
new_test_data.head()

In [ ]:
#Skewness After Transformation:
new_test_data.skew()

# Apply Model to the Test Data for Predicting The item Outsale Values:

In [ ]:
item_outsale_pred = regressor.predict(new_test_data)

In [ ]:
item_outsale_pred

## **Above predicted values are transformed values. Therefore, we need to inverse transform it to get actual predicted values.** 

In [ ]:
#Performing inverse transformation
actual_item_outsale = np.exp(item_outsale_pred+1)

In [ ]:
actual_item_outsale

In [ ]:
#ADDING THE PREDICTED ITEM_OUTLET_SALE COLUMNS TO TEST DATA
test["item_outlet_sale"] = actual_item_outsale

In [ ]:
test